# LangGraph를 사용한 고객 지원 챗봇을 구축

- 웹 검색을 통해 일반적인 질문에 답변 
- 대화 상태를 유지하여 연속적인 대화  
- 복잡한 질문을 사람이 검토하도록 라우팅  
- 사용자 지정 상태(Custom State)를 활용하여 챗봇의 동작 제어  
- 대화 흐름을 되돌리고(Rewind), 다른 대화 경로 탐색 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적 설정 활성화
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano")

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

## 도구를 활용한 챗봇 강화**   
이제 챗봇이 **웹에서 관련 정보를 찾아 더 나은 답변을 제공할 수 있도록 개선**합니다. 

---
먼저 **Tavily 검색 엔진**을 사용하기 위해 필요한 패키지를 설치하고 **`TAVILY_API_KEY`** 를 설정 합니다.  

In [4]:
# %%capture --no-stderr
#%pip install -U langchain-tavily

In [5]:
# Tavily 검색 툴을 임포트
# Tavily는 웹 검색을 통해 정보를 가져오는 도구로, LangChain에서 도구(tool)로 활용 가능
from langchain_tavily import TavilySearch

# TavilySearch 툴 인스턴스를 생성 (최대 결과 2개로 제한)
tool = TavilySearch(max_results=2)

# 사용할 툴들을 리스트로 구성 (여러 개의 도구가 필요한 경우를 대비해 리스트 형태로 작성)
tools = [tool]

# Tavily 검색 도구를 직접 호출하여 "LangGraph에서 node가 뭐야?"라는 질문에 대한 웹 검색 결과를 가져옴
result = tool.invoke("LangGraph에서 node가 뭐야?")
print(result)

{'query': 'LangGraph에서 node가 뭐야?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'LangGraph의 핵심 구조: Node & Edge : 네이버 블로그', 'url': 'https://blog.naver.com/oaziz/223839790474', 'content': 'LangGraph의 핵심 구조: Node & Edge 1. 🧱 Node (노드): 작업 단위 (작업자 혹은 처리 단계) 🎯 정의 하나의 노드는 하나의 기능 또는 하나의 처리 단계를 담당합니다. 예: LLM 호출, retriever 실행, 요약, 사용자 입력 처리 등 🛠 Node에는 보통 다음과 같은 함수가 들어갑니다:', 'score': 0.8719229, 'raw_content': None}, {'title': 'Nodes and Edges | langchain-ai/langgraph-101 | DeepWiki', 'url': 'https://deepwiki.com/langchain-ai/langgraph-101/2.2-nodes-and-edges', 'content': 'Nodes and Edges | langchain-ai/langgraph-101 | DeepWiki Nodes and Edges Nodes and Edges What are Nodes and Edges? In LangGraph, a graph is composed of nodes connected by edges to form a directed workflow. Nodes are the workhorses of LangGraph - they are Python functions that receive the current graph state as input, perform operations, and return updates to that state. Edges define the flow o

In [6]:
# pip install -U "langchain[openai]"

### 에이전트 생성하기
이제 도구들과 LLM(언어 모델)을 정의했으니, 에이전트를 생성할 수 있습니다. 우리는 **LangGraph**를 사용하여 에이전트를 구성할 것입니다. 현재는 **상위 수준의 인터페이스**를 사용하여 에이전트를 만들고 있지만, LangGraph의 장점은 이 상위 수준 인터페이스가 **하위 수준의 고도로 제어 가능한 API**로 지원된다는 점입니다. 따라서 나중에 에이전트 로직을 자유롭게 수정할 수 있습니다.

에이전트는 세 가지 구성 요소로 이루어져 있습니다:
**대규모 언어 모델(LLM)**, 사용할 수 있는 **도구들의 집합**, 그리고 **지시사항을 담은 프롬프트**입니다.

LLM은 루프 방식으로 작동합니다. 각 반복(iteration)마다 다음과 같은 과정을 수행합니다:

1. 사용할 도구를 선택하고,
2. 그 도구에 입력을 제공하며,
3. 결과(관찰값, observation)를 받아오고,
4. 그 관찰값을 바탕으로 다음 행동을 결정합니다.

이 루프는 **중지 조건**이 충족될 때까지 계속되며, 일반적으로는 **사용자에게 응답하기에 충분한 정보를 수집했을 때** 종료됩니다.

In [7]:
# Import relevant functionality
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [8]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "안녕, 난 길동이야. 지금 서울의 날씨가 어때?",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

안녕, 난 길동이야. 지금 서울의 날씨가 어때?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_kNO3JvzpOMPVGnJVQDtucL4R)
 Call ID: call_kNO3JvzpOMPVGnJVQDtucL4R
  Args:
    query: 서울 오늘 날씨
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "서울 오늘 날씨", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://weather.com/ko-KR/weather/today/l/18e81cdf57491c51a6fba3c57732b7b61bdf511fc2b613570316978b9f20687a", "title": "서울특별시 일기예보 및 날씨 - The Weather Channel", "content": "오늘 서울특별시의 날씨 예보 ; 최고/최저. --/20° ; 바람. 6 km/h ; 습도. 61% ; 이슬점. 17° ; 기압. 1012.2 mb.", "score": 0.8235701, "raw_content": null}, {"url": "https://www.accuweather.com/ko/kr/seoul/226081/hourly-weather-forecast/226081", "title": "서울특별시, 서울시, 대한민국 시간별 날씨 - AccuWeather", "content": "서울특별시

## 챗봇에 메모리 기능 추가¶**  

현재 챗봇은 **사용자 질문에 도구를 활용해 답변할 수 있지만, 이전 대화의 맥락을 기억하지 못합니다.**  
이 때문에 **일관된 멀티턴(Multi-turn) 대화를 진행하는 데 한계가 있습니다.**  

LangGraph는 **"지속적 체크포인트(Persistent Checkpointing)"** 기능을 통해 이 문제를 해결합니다.  

그래프를 컴파일할 때 checkpointing을 활성화하고 그래프를 호출할 때 `thread_id`를 제공하면, LangGraph가 자동으로 상태(state)를 저장하고, 다음 실행 시 이전 상태를 복원합니다.  

즉, **동일한 `thread_id`** 를 사용하여 그래프를 호출하면, 이전 대화 상태를 불러와서 이어서 대화할 수 있습니다!

In [9]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

우리는 현재 **메모리를 활용하는(in-memory) 체크포인터**를 사용하고 있습니다.  

이 방식은 튜토리얼 환경에서는 편리하지만, 데이터가 메모리에만 저장되므로 영구적이지 않습니다. 실제 프로덕션 환경에서는 `SqliteSaver` 또는 `PostgresSaver`를 사용하여 데이터베이스(DB)와 연결하는 것이 일반적입니다.

---

In [10]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

input_message = {"role": "user", "content": "안녕, 난 길동이야."}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

안녕, 난 길동이야.
================================== Ai Message ==================================

안녕, 길동아! 어떻게 도와줄까?


### 무한 loop 로 Chatbot 구현

In [11]:
from langchain_core.messages import AIMessage

# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
memory = MemorySaver()

# 그래프를 메모리 체크포인트와 함께 컴파일합니다.
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc345"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    for step in agent_executor.stream({"messages": [user_input]}, config, stream_mode="values"):
        step["messages"][-1].pretty_print()

User:  안녕 난 길동이야


================================ Human Message =================================

안녕 난 길동이야
================================== Ai Message ==================================

안녕하세요, 길동님! 어떻게 도와드릴까요?


User:  지금 서울 날씨는?


================================ Human Message =================================

지금 서울 날씨는?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_Pg09T7HOIOj4v0LD0lvNwjb1)
 Call ID: call_Pg09T7HOIOj4v0LD0lvNwjb1
  Args:
    query: 서울 현재 날씨
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "서울 현재 날씨", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/ko/kr/seoul/226081/current-weather/226081", "title": "서울특별시, 서울시, 대한민국 현재 날씨 - AccuWeather", "content": "서울특별시, 서울시, 대한민국 현재 날씨 | AccuWeather 서울특별시, 서울시 ========== 65°F 현재 위치 사용 서울특별시 서울시 65° 서울특별시, 서울시 날씨 오늘 WinterCast 지역 {stormName} 추적기 시간별 일별 레이더 MinuteCast 월 대기질 건강 및 활동 오늘 시간별 일별 레이더 MinuteCast 월 대기질 건강 및 활동 현재 기상 RealFeel® 71° RealFeel Shade™ 63° 71° 63° 최대 자외선 지수 서 5mi/h 5mi/h 48° F 5mi RealFeel® 81° RealFeel Shade™ 73° 최대 자외선 지수10 

User:  내 이름이 뭐지?


================================ Human Message =================================

내 이름이 뭐지?
================================== Ai Message ==================================

길동님이세요! 다른 궁금한 점 있으시면 언제든 말씀해 주세요.


User:  q


Goodbye!
